<div style="text-align: center;">
<h1>CS 614 Algorithms in Bioinformatics - Fall 2020</h1>
<h1 >Assignment #3 </h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late assignments will be penalized by a 10% per day. No homework will be accepted more than one week late. Feel free to ask for questions and help, but **the work you submit in should be your own.** It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded.

In [2]:
# Replace the following string values with the requested information
# Leave the quotation marks surronding the string!
class Student:
    first = "firstName"
    last = "lastName"
    sid = "StudentID"

<h3> Problem 1 Motif profile (10 points) </h3>
<p>
The following 10 sequences contain a motif of length 8:   
<p style="font-family: Courier,monospace;">
GAACTCATGGTG<br>
AAAAGCACGGTC<br>
TCAAAGCAAGGC<br>
CCTAATCAGGGC<br>
AAGTATGGACTC<br>
ACTAAGCAGGGT<br>
TCTCACGGCCCA<br>
CCTCGTGGTGGG<br>
TACCGTATGGTT<br>
ACCACTCGTCGA<br>
</p>
<p>

<h5> Problem 1.1 </h5>
<p>
Find the highest-score motif of length 8 in the 10 sequences shown above. Construct an alignment of all instances of the motif and compute the profile and consensus sequence of the motif (In the event of a “tie”, select one nucleotide as a representative for that position).
    
<p>
Hint: you may use the functions in the example ipynb's in the class. 

In [ ]:
#This cell is provided for testing your code. 


<h5>Enter your answers here. THIS CELL WILL BE GRADED. </h5>

What is the alignment of all instances of the motif?

<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>

What is the profile of this motif?

<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>

What is the consensus of this motif?
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>

What is the entropy score of the motif profile?
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>

<h5>Problem 1.2</h5>
Compute the likelihood ratio of getting “ACTCATGG” according to motif profile vs. a background model of each base having equal probability of 25%. (Show the steps or code)
<p>
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>

<h3> Problem 2 Sequence Assembly (15 points)</h3>
In this assignment, you will try out sequence assembly using Eulerian path algorithms respectively. You will need a set of 15 base-pair simulated reads from a genome that will be used for assembly. The cell below provides various functions and imports necessary for this problem set. Do not import any addtional packages. Also, make sure that you run the following cell. 

In [2]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(fp):
    """ Parses a classically formatted FASTA file into two lists. 
        One of headers and a second list of sequences.
        The ith index of each list correspond."""
    # split at headers
    data = fp.read().decode('utf-8').split('>')
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        sequences.append(''.join(lines))
    return (headers, sequences)

class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
        
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
        
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    
    def hamiltonianPath(self):
        """ A Brute-force method for finding a Hamiltonian Path. 
        Basically, all possible N! paths are enumerated and checked
        for edges. Since edges can be reused there are no distictions
        made for *which* version of a repeated edge. """
        for path in itertools.permutations(sorted(self.index.values())):
            for i in range(len(path)-1):
                if ((path[i],path[i+1]) not in self.edge):
                    break
            else:
                return [self.vertex[i] for i in path]
        return []
    
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    
    def hamiltonianPathV2(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result
    
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.keys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
        
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print("There is no path!")
                    return False
        vpath = [self.vertex[i] for i in path]
        return vpath
    
    def eulerEdges(self, path):
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in range(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList
    
    def render(self, highlightPath=[]):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeSet = set()
        for i in range(len(highlightPath)-1):
            src = self.index[highlightPath[i]]
            dst = self.index[highlightPath[i+1]]
            edgeSet.add(edgeId[src,dst].pop())
        result = ''
        result += 'digraph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for index, label in self.vertex.items():
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (index, label)
        for i, e in enumerate(self.edge):
            src, dst = e
            result += '    N%d -> N%d' % (src, dst)
            label = self.edgelabel[i]
            if (len(label) > 0):
                if (i in edgeSet):
                    result += ' [label="%s", penwidth=3.0]' % (label)
                else:
                    result += ' [label="%s"]' % (label)
            elif (i in edgeSet):
                result += ' [penwidth=3.0]'                
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result

<h5> Problem 2.1 </h5>
A file of 15-mers simulating short reads file <a href="https://raw.githubusercontent.com/xiaoyu12/CS614_Jupyter_notebooks/master/data/reads.fa" download="reads.fa">reads.fas</a> from a genome. Construct a De Bruijin graph using these 15-mers as edges, and assemble the superstring using the eulerianPath() function. Answer the following questions based on the De Bruijin graph and assembly.

In [5]:
# This cell is provided for your own use in answering problem 2. It will not be graded.
# Read an example genome from an URL
import urllib.request

url = "https://raw.githubusercontent.com/xiaoyu12/CS614_Jupyter_notebooks/master/data/reads.fa"
with urllib.request.urlopen(url) as fp:
    headers, reads = loadFasta(fp)
    
# Use the Graph class defined above and add your own code to answer the questions below. 


<h5>Enter your answers here. THIS CELL WILL BE GRADED. </h5>
1. How many distinct nodes appear in the De Bruijn graph that represents these 15-mers as edges? 
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>
2. How many nodes are semi-balenced? How many nodes are balanced? 
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>   
3. How many are balanced with both in-degrees and out-degrees equal to 1?
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>
4. What is the length of assembled superstring following the Eulerian path that can be constructed in the De Bruijn graph?
    
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>
5. The assembled superstring should be from the genome <a href="https://raw.githubusercontent.com/xiaoyu12/CS614_Jupyter_notebooks/master/data/sequence.fa" download="sequence.fa">sequence.fa</a>. What is the starting position of this assembled sequence in that genome? Is the assembled sequence on the forward or the reverse-complement strand of the genome?

<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>

<h3> Problem 3 (15 points)</h3>
In this assignment, you will implement the Gibbs Sampler for motif finding in the GibbsMotifSearch() function. 

<p>
    Make sure run the following cell, which contains useful functions.

In [1]:
import numpy as np

def Profile(s, DNA, k):
    """ Compute the profile of k-mer motifs in a DNA string 
        s = list of starting indices, 1-based, 0 means ignore, DNA = list of nucleotide strings,
        k = Target Motif length"""
    profile = np.zeros((4, k))
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        profile[:, i] = np.fromiter(cnt.values(), dtype=float) / (float)(len(s))
    return profile

def Score(s, DNA, k):
    """ compute the consensus SCORE of a given k-mer alignment given 
        offsets into each DNA string. s = list of starting indices.
        DNA = list of nucleotide strings. k = Target Motif length
    """
    score = 0
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            base = DNA[j][sval+i] 
            cnt[base] += 1
        score += max(cnt.values())
    return score


def LikelyhoodKmerProfile(kmer, profile):
    """Given a profile, compute the probability of a kmer string"""
    # check that kmer length matches with profile length
    assert len(kmer) == profile.shape[1]
    prob = 1.0
    idx = dict(zip("acgt", (0, 1, 2, 3)))
    for i in range(len(kmer)):
        prob = prob * profile[idx[kmer[i]], i]
    return prob

def ScanAndScoreProfile(DNA, profile):
    """ Given a list of sequences in DNA and a profile,
    find the best alignments of the profile in all sequences"""
    bestAlignment = []
    k = profile.shape[1] 
    for seq in DNA:
        maxProbProfile = 0
        for s in range(len(seq)-k+1):
            prob = ScoreKmerProfile(seq[s:s+k], profile)
            if (prob > maxProbProfile):
                bestS = s
                maxProbProfile = prob
        bestAlignment.append(bestS)
    return bestAlignment

def Consensus(s, DNA, k):
    """ compute the consensus k-Motif of an alignment given offsets into each DNA string.
            s = list of starting indices, 1-based, 0 means ignore, DNA = list of nucleotide strings,
            k = Target Motif length """
    consensus = ''
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        consensus += max(cnt.items(), key=lambda tup: tup[1])[0]
    return consensus

In [3]:
# Problem 3 function. This cell is graded.
# You can use functions defined in the previous cell

import numpy as np
from numpy.random import randint
from numpy.random import choice

def GibbsMotifSearch(DNA, k, r = 30, N = 5000):
    """ Searches for a k-length motif that appears 
    in all given DNA sequences. It begins with a
    random set of candidate consensus motifs 
    derived from the data. It refines the motif
    using Gibbs sampler until a true consensus emerges.
    DNA - list of DNA sequences
    k - length of kmer motif
    r - number of random initial alignments
    N - number of maximum iterations for each Gibbs sampling
    """
    
    # initialization
    bestAlignment = []
    bestScore = 0 
    
    for sample in range(r):    # Try r different random initial alignments
        # Choose a random initial alignment
        #   ~~ Your code goes here~~

        n = len(DNA)      # number of sequences
        for i in range(N):     # Repeat the Gibbs sampling N steps
            # select a random sequence for updating. Hint: use the numpy randint() function
            #   ~~ Your code goes here~~
            
            # compute the profile of motifs except for the selected sequence
            #   ~~ Your code goes here~~
            
            # Use the profile to compute the likelyhoods of all k-mers in the select sequence
            #   ~~ Your code goes here~~
            
            # normalize the likelyhoods to compute the sampling probablitiy 
            # Note: try to check for divide by zero error. 
            # If total likelyhood is 0, assign equal probability to k-mer positions
            #   ~~ Your code goes here~~
            
            # sample a position in the selected sequence using the probability
            # Hint: use the numpy choice() function
            #   ~~ Your code goes here~~
            
            # update the alignment using the sampled position
            #   ~~ Your code goes here~~

            # compute the score of the new alignment 
            # if the alignment has a better score than the current best alignment, update the bestAlignment 
            #   ~~ Your code goes here~~
            
    
    bestScore = Score(bestAlignment, DNA, k)
    
    return bestAlignment, bestScore
    
    

In [ ]:
# Test your function in this cell
seqApprox = [
    'tagtggtcttttgagtgtagatctgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttggatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagacccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtaaatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattaggtccaaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagattcgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgcagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgta',
    'ttcttacacccttctttagatccaaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtcgatccgaaattcg']

bestAlignment, bestScore = GibbsMotifSearch(seqApprox, 10, 30, 5000)

# Compare it with the result from  MedianMotifSearch()

Compare your result with that from the MedianMotifSearch(). Are they the same?
<p> <span style="color: red"> ~~ Your answer goes here ~~ </span> </p>